# Análisis de datos

In [30]:
from functools import reduce
import re
import math

In [10]:
with open("borrador_nueva_constitución.txt", 'r') as file:
    text = file.read()
    arts = re.split("-"*50, text)
print(len(arts))

499


Largo del artículo más largo:

In [21]:
max_art = max(arts, key=len)
print("Artículo:", max_art.split(".-")[0].strip())
print("Largo:", len(max_art))
print("Tweets aproximados:", math.ceil(len(max_art) / 269))

Artículo: 441
Largo: 4579
Tweets aproximados: 17


Artículo con más incisos:

In [23]:
arts_splited = (art.strip().split('\n') for art in arts)
max_art = max(arts_splited, key=len)
print("Artículo:", max_art[0].split(".-")[0].strip())
print("Incisos:", len(max_art))

Artículo: 186
Incisos: 27


Artículo que requiere más tweets:

In [46]:
def n_tweets(art):
    art_splited = art.strip().split('\n')
    n = reduce(lambda a, b: a+b, (math.ceil(len(inc) / 269) for inc in art_splited))
    return n

In [47]:
max_art = max(arts, key=n_tweets)
print("Artículo:", max_art.split(".-")[0].strip())
print("Tweets:", n_tweets(max_art))

Artículo: 186
Tweets: 29
